In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1'

embedding_model = SentenceTransformer(model_name)

In [2]:
user_question = "I just discovered the course. Can I still join it?"

res = embedding_model.encode(user_question)

res[0]

0.07822264

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
documents = [item for item in documents if item["course"] == "machine-learning-zoomcamp"]

In [4]:
import numpy as np

embeddings = [
    embedding_model.encode(f'{doc["question"]} {doc["text"]}') for doc in documents
]

for doc, embedding in zip(documents, embeddings):
    doc["qa_text"] = embedding

X = np.array(embeddings)

X.shape


(375, 768)

In [5]:
scores = X.dot(res)
max(scores)

0.6506573

In [7]:
import numpy as np
from typing import List, Dict, Any

class VectorSearchEngine:
    def __init__(self, documents: List[Dict[str, Any]], embeddings: np.ndarray):
        """
        Initializes the VectorSearchEngine with documents and their embeddings.

        :param documents: List of documents where each document is a dictionary.
        :param embeddings: NumPy array of embeddings corresponding to the documents.
        """
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query: np.ndarray, num_results: int = 10) -> List[Dict[str, Any]]:
        """
        Searches for the most similar documents to the given query vector.

        :param v_query: The query vector to search for.
        :param num_results: The number of top results to return.
        :return: A list of the most similar documents.
        """
        if v_query.shape[0] != self.embeddings.shape[1]:
            raise ValueError("Query vector dimension does not match embedding dimension.")
        
        scores = self.embeddings.dot(v_query)
        idx = np.argpartition(-scores, num_results)[:num_results]
        sorted_idx = idx[np.argsort(-scores[idx])]
        return [self.documents[i] for i in sorted_idx]

# Usage
search_engine = VectorSearchEngine(documents, X)
results = search_engine.search(res, num_results=5)
results


In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from urllib.parse import urljoin
from elasticsearch import Elasticsearch
from typing import List

# Load and filter ground truth data
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

search_engine = VectorSearchEngine(documents, X)

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v = embedding_model.encode(q['question'])
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

print(hit_rate(relevance_total))

100%|██████████| 1830/1830 [04:47<00:00,  6.36it/s]

0.9398907103825137


In [9]:
from elasticsearch import Elasticsearch
from tqdm.autonotebook import tqdm 

es_client = Elasticsearch("htttp://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = 'course-questions'

if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
else:
    print(f"Index '{index_name}' does not exist.")

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

Index 'course-questions' deleted.


100%|██████████| 375/375 [00:02<00:00, 146.59it/s]


In [10]:
from typing import List, Dict

def elastic_search_knn(field: str, vector: List[float], course: str) -> List[Dict[str, any]]:
    """
    Perform a KNN search on Elasticsearch and return the result documents.

    :param field: The field to search in.
    :param vector: The query vector.
    :param course: The course to filter by.
    :return: A list of dictionaries containing the search results.
    """
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(index=index_name, body=search_query)

    return [hit['_source'] for hit in es_results['hits']['hits']]


In [11]:
user_question = "I just discovered the course. Can I still join it?"
res = embedding_model.encode(user_question)
elastic_search_knn("qa_text", res, 'machine-learning-zoomcamp')

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [12]:
def question_vector_knn(q, search_engine, model):
    question = q['question']
    res = model.encode(question)
    return search_engine("qa_text", res, 'machine-learning-zoomcamp')

def hit_rate(relevance_total):
    hits = sum(1 for relevance in relevance_total if any(relevance))
    return hits / len(relevance_total)

def evaluate(ground_truth, search_function, search_engine, model):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q, search_engine, model)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

result = evaluate(ground_truth, question_vector_knn, elastic_search_knn, embedding_model)

print(f"Hitrate: {result['hit_rate']}")


100%|██████████| 1830/1830 [01:53<00:00, 16.16it/s]

Hitrate: 0.9398907103825137
